# Projeto Spark

## Introdução

Neste projeto vamos construir um classificador Naive-Bayes para determinar o sentimento de um comentário.

## Instalando o ambiente

O jeito mais simples de começar a trabalhar com Spark é instalar um container com tudo pronto! No site https://hub.docker.com/r/jupyter/pyspark-notebook vemos uma imagem Docker que já vem com `pyspark` e `jupyter lab`. Instale a imagem com o comando:

```bash
docker pull jupyter/pyspark-notebook
```

Vamos iniciar o ambiente de trabalho com o comando `docker run`. Para isso precisamos tomar alguns cuidados:

1) Temos que mapear nosso diretorio local de trabalho para um diretório interno do container, de modo que alterações feitas dentro do container (nesta pasta escolhida) sejam gravadas no nosso diretorio local. No container temos um usuário padrão com *username* `jovyan`. No *homedir* desse usuario temos uma pasta vazia `work`, que vai servir como local de mapeamento do nosso diretorio local de trabalho. Podemos então fazer esse mapeamendo com a opção `-v` do comando `docker run` da seguinte forma:

```bash
-v <diretorio>:/home/jovyan/work
```

onde `<diretorio>` representa seu diretorio local de trabalho.

2) Para acessar o `jupyter notebook` e o *dashboard* do Spark a partir do nosso *browser* favorito temos que abrir algumas portas do container com a opção `-p`. As portas são `8888` (para o próprio `jupyter notebook`) e `4040` (para o *dashboard* do Spark). Ou seja, adicionaremos às opções do `docker run`o seguinte:

```bash
-p 8888:8888 -p 4040:4040
```

Desta forma, ao acessar `localhost:8888` na nossa máquina, estaremos acessando o servidor Jupyter na porta 8888 interna do container.

3) Vamos iniciar o container no modo interativo, e vamos especificar que o container deve ser encerrado ao fechar o servidor Jupyter. Faremos isso com as opções `-it` e `-rm`

Portanto, o comando completo que eu uso na minha máquina Linux para iniciar o container é:

```bash
docker run \
    -it \
    --rm \
    -p 8888:8888 \
    -p 4040:4040 \
    -v "`pwd`":/home/jovyan/work \
    jupyter/pyspark-notebook
```

Para facilitar a vida eu coloco esse comando em um arquivo `inicia.sh`. Engenheiros, façam do jeito que preferirem!

Agora abra esse notebook lá no container!


## Iniciando o Spark

Vamos iniciar o ambiente Spark. Para isso vamos:

1) Criar um objeto de configuração do ambiente Spark. Nossa configuração será simples: vamos especificar que o nome da nossa aplicação Spark é "Minha aplicação", e que o *master node* é a máquina local, usando todos os *cores* disponíveis. Aplicações reais de Spark são configuradas de modo ligeiramente diferente: ao especificar o *master node* passamos uma URL real, com o endereço do nó gerente do *cluster* Spark.

2) Vamos criar um objeto do tipo `SparkContext` com essa configuração

In [1]:
import pyspark

conf = pyspark.SparkConf()
conf.setAppName('Minha aplicação')
conf.setMaster('local[*]')

sc = pyspark.SparkContext(conf=conf)

O `SparkContext` é a nossa porta de entrada para o cluster Spark, ele será a raiz de todas as nossas operações com o Spark.

In [2]:
sc

<SparkContext master=local[*] appName=Minha aplicação>

O link acima provavelmente não funcionará porque ele se refere à porta 4040 interna do container (portanto a URL está com endereço interno). Porém fizemos o mapeamento da porta 4040 interna para a porta 4040 externa, logo você pode acessar o *dashboard* do Spark no endereço http://localhost:4040

<center><img src="./spark_dashboard.png" width=800/></center>

## Lendo os dados

Vamos começar lendo o arquivo de reviews e gravando o resultado em formato pickle, mais amigável.

In [4]:
def parse_line(line):
    parts = line[1:-1].split('","')
    sentiment = int(parts[0])
    title = parts[1].replace('""', '"')
    body = parts[2].replace('""', '"')
    return (sentiment, title, body)

rdd = sc.textFile('train.csv').map(parse_line)

In [5]:
rdd.count()

3600000

In [11]:
rdd.take(1)

[(2,
  'Stuning even for the non-gamer',
  'This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^')]

Agora vamos gravar no formato pickle, para facilitar os trabalhos futuros. Após gravar o arquivo, não mais rode as células desta primeira etapa!

In [8]:
rdd.saveAsPickleFile('reviews.pickle')

## Um classificador Naive-Bayes

Vamos ler o arquivo pickle gravado anteriormente:

In [8]:
rdd = sc.pickleFile('reviews.pickle')

In [9]:
rdd.count()

3600000

In [12]:
# rdd.take(10)

Agora, complete as tarefas em sequencia para construir o classificador Naive-Bayes:

### Fase 1

#### Tarefa

Construa uma função que recebe um RDD no formato do RDD original e retorna um RDD no qual cada item é um par (palavra, contagem).

In [13]:
def count_words(rdd):
    word_counts = rdd.flatMap(lambda line: line[2].split(" ")) \
                    .countByValue()
    return sc.parallelize(list(word_counts.items()))


    
word_counts_rdd = count_words(rdd)

# for word, count in word_counts_rdd.collect():
    # print(f"{word}: {count}")

#### Tarefa

Construa uma função que recebe o RDD (palavra, contagem) construido anteriormente e retorna um RDD no qual cada item é um par (palavra, $\log_{10}\left(c \, / \, T\right)$), onde $c$ é a contagem daquela palavra e $T$ é a soma das contagens de palavra.

In [14]:
import math

def calculate_log_tf(rdd):
    total_count = rdd.map(lambda x: x[1]).sum()
    log_tf_rdd = rdd.map(lambda x: (x[0], math.log(x[1] / total_count)))

    return log_tf_rdd
    
log_rdd = calculate_log_tf(word_counts_rdd)
# log_rdd.take(3)

#### Tarefa

Separe o RDD original em dois RDDs: o dos reviews positivos e o dos negativos. Em seguida, use as funções anteriores para construir RDDs que contem os pares (palavra, $\log_{10}\left(c \, / \, T\right)$)

In [19]:
positive_reviews_rdd = rdd.filter(lambda x: x[0] == 2)
negative_reviews_rdd = rdd.filter(lambda x: x[0] == 1)

positive_word_count_rdd = count_words(positive_reviews_rdd)
positive_log_rdd = calculate_log_tf(positive_word_count_rdd)

negative_word_count_rdd = count_words(negative_reviews_rdd)
negative_log_rdd = calculate_log_tf(negative_word_count_rdd)

### Tarefa

Use o `.fullOuterJoin()` dos RDDs para construir um RDD unificado, no qual cada item é da forma (palavra, log_prob_positivo, log_prob_negativo). "Baixe" esse resultado final usando `.collect()`.

In [24]:
unified_rdd = positive_log_rdd.fullOuterJoin(negative_log_rdd)

result = unified_rdd.collect()

[('out', (-6.095643264455923, -5.955354168570672)),
 ('of', (-3.8723180279942113, -3.967716609256227)),
 ('ever', (-7.192057586482426, -7.25216531117948)),
 ('grate', (-12.556924339049992, -12.43896181187521)),
 ('guitars', (-10.799372302053646, -11.511101898178284)),
 ('listening', (-8.464392209126661, -8.942399155507749)),
 ('fade.The', (-18.66839167855267, -18.74888009010173)),
 ('are', (-5.151299285856768, -5.247280596806589)),
 ('years.', (-8.574945142093377, -8.915868651369923)),
 ('like', (-5.7155386727431035, -5.677399565012032)),
 ('(indefinably', (-18.66839167855267, None)),
 ('Burning', (-12.684455397865479, -13.110525420767981)),
 ('excellent', (-7.478913642480965, -9.073171817284337)),
 ('heart-rendering', (-15.778019920656506, -18.05573290954178)),
 ('everything', (-8.015825673135799, -8.32430969934696)),
 ('Home),', (-16.722481529497358, -17.650267801433618)),
 ('God),', (-15.449515853684469, -15.490783552080245)),
 ('raving', (-12.586172768176224, -11.901936950516347)),

#### Tarefa

Para uma dada string, determine se ela é um review positivo ou negativo usando os RDDs acima. Lembre-se de como funciona o classificador Naive-Bayes: http://stanford.edu/~jurafsky/slp3/slides/7_NB.pdf, consulte tambem suas notas de aula de Ciência dos Dados!

In [30]:
def classify_review(review_string):

    total_log_prob_positivo = 0.0
    total_log_prob_negativo = 0.0

    for palavra in review_string.split():
        log_prob_positivo = positive_log_rdd.lookup(palavra)  
        log_prob_negativo = negative_log_rdd.lookup(palavra) 

        if log_prob_positivo and log_prob_negativo:
            total_log_prob_positivo += log_prob_positivo[0]
            total_log_prob_negativo += log_prob_negativo[0]

    if total_log_prob_positivo > total_log_prob_negativo:
        return "Review positivo"
    else:
        return "Review negativo"

review = "This movie is fantastic! The acting is kind good but the plot is ok."
classification = classify_review(review)
print(classification)

Review positivo


### Fase 2

Agora que temos um classificador Naive-Bayes, vamos explorá-lo um pouco:

### Tarefa

Quais são as 100 palavras que mais indicam negatividade, ou seja, onde a diferença entre a probabilidade da palavra no conjunto dos comentários negativos e positivos é máxima? E quais as 100 palavras de maior positividade? Mostre os resultados na forma de *word clouds*.

In [ ]:
from wordcloud import WordCloud

import matplotlib.pyplot as plt

# Calculate the difference between log probabilities
def calculate_difference(unified_rdd):
    return unified_rdd.map(lambda x: (x[0], (x[1][0] if x[1][0] is not None else 0) - (x[1][1] if x[1][1] is not None else 0)))

difference_rdd = calculate_difference(unified_rdd)

# Get the top 100 words for negativity and positivity
top_negative_words = difference_rdd.takeOrdered(100, key=lambda x: x[1])
top_positive_words = difference_rdd.takeOrdered(100, key=lambda x: -x[1])

# Convert to dictionary for word cloud
negative_word_dict = dict(top_negative_words)
positive_word_dict = dict(top_positive_words)

# Generate word clouds
def generate_wordcloud(word_dict, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_dict)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

generate_wordcloud(negative_word_dict, 'Top 100 Negative Words')
generate_wordcloud(positive_word_dict, 'Top 100 Positive Words')

### Tarefa desafio!

Qual o desempenho do classificador (acurácia)? Para medir sua acurácia:

- Separe os reviews em dois conjuntos: treinamente e teste
- Repita o "treinamento" do classificador com o conjunto de treinamento
- Para cada review do conjunto de teste, determine se é positiva ou negativa de acordo com o classificador
- Determine a acurácia

Esta não é uma tarefa trivial. Não basta fazer um `for` para determinar a classe de cada review de teste: isso demoraria uma eternidade. Você tem que usar variáveis "broadcast" do Spark para enviar uma cópia da tabela de frequencias para cada *core* do executor.

### Tarefa desafio!

Implemente Laplace smoothing

In [ ]:
def calculate_log_tf_laplace(rdd, vocab_size, alpha=1.0):
    total_count = rdd.map(lambda x: x[1]).sum()
    log_tf_rdd = rdd.map(lambda x: (x[0], math.log((x[1] + alpha) / (total_count + alpha * vocab_size))))
    return log_tf_rdd

# Calculate the vocabulary size
vocab_size = word_counts_rdd.count()

# Recalculate log probabilities with Laplace smoothing
positive_log_rdd_laplace = calculate_log_tf_laplace(positive_word_count_rdd, vocab_size)
negative_log_rdd_laplace = calculate_log_tf_laplace(negative_word_count_rdd, vocab_size)

def classify_review_laplace(review_string):
    total_log_prob_positivo = 0.0
    total_log_prob_negativo = 0.0

    for palavra in review_string.split():
        log_prob_positivo = positive_log_rdd_laplace.lookup(palavra)
        log_prob_negativo = negative_log_rdd_laplace.lookup(palavra)

        if log_prob_positivo:
            total_log_prob_positivo += log_prob_positivo[0]
        else:
            total_log_prob_positivo += math.log(1 / (positive_word_count_rdd.map(lambda x: x[1]).sum() + vocab_size))

        if log_prob_negativo:
            total_log_prob_negativo += log_prob_negativo[0]
        else:
            total_log_prob_negativo += math.log(1 / (negative_word_count_rdd.map(lambda x: x[1]).sum() + vocab_size))

    if total_log_prob_positivo > total_log_prob_negativo:
        return "Review positivo"
    else:
        return "Review negativo"

# Test the classifier with Laplace smoothing
review = "This movie is fantastic! The acting is kind good but the plot is ok."
classification = classify_review_laplace(review)
print(classification)